count table -> deseq

In [1]:
import pandas as pd
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

# 샘플 리스트 및 조건 매핑
samples = ["SRR5292039", "SRR5292040", "SRR5292041", "SRR5292042", 
           "SRR5292043", "SRR5292044", "SRR5292045", "SRR5292046", 
           "SRR5292047", "SRR5292048", "SRR5292049", "SRR5292050", 
           "SRR5292051", "SRR5292052", "SRR5292053", "SRR5292054"]

conditions = ["TBI-Acute-Control", "TBI-Acute-Control", "TBI-Acute-Control", 
              "TBI-Acute", "TBI-Acute", "TBI-Acute",
              "TBI-Subacute-Control", "TBI-Subacute-Control", 
              "TBI-Subacute", "TBI-Subacute", "TBI-Subacute", 
              "TBI-Chronic-Control", "TBI-Chronic-Control", 
              "TBI-Chronic", "TBI-Chronic", "TBI-Chronic"]

# 카운트 데이터를 하나의 DataFrame으로 결합
count_data = pd.DataFrame()

for sample in samples:
    sample_data = pd.read_csv(f"/data1/project/eunyi/tbi/data/htseq_counts/{sample}_counts.txt", sep="\t", header=None, index_col=0)
    
    # 메타데이터 행 제거
    sample_data = sample_data[~sample_data.index.str.startswith("__")]
    
    count_data[sample] = sample_data[1]

# gene 열을 인덱스로 설정
count_data.index.name = "gene"

# DESeq2 분석을 위한 함수 정의
def run_deseq2_analysis(count_data, col_data, condition1, condition2):
    # 해당 조건에 해당하는 샘플만 선택
    selected_samples = col_data[(col_data['condition'] == condition1) | (col_data['condition'] == condition2)].index
    selected_count_data = count_data[selected_samples]
    selected_col_data = col_data.loc[selected_samples]
    
    # DESeq2 분석 수행
    dds = DeseqDataSet(counts=selected_count_data.T, metadata=selected_col_data, design_factors="condition", refit_cooks=True, n_cpus=8)
    dds.deseq2()
    deseq_stats = DeseqStats(dds)
    deseq_stats.summary()
    
    # 결과 반환
    return deseq_stats.results_df[['log2FoldChange', 'pvalue', 'padj']]

# 샘플 조건을 DataFrame으로 변환
col_data = pd.DataFrame({'condition': conditions}, index=samples)
col_data['condition'] = pd.Categorical(col_data['condition'])

# TBI Acute vs Acute Control 분석
acute_results = run_deseq2_analysis(count_data, col_data, "TBI-Acute", "TBI-Acute-Control")
acute_results.columns = ["TBI Acute vs AControl Log2 fold change", "Acute pval", "Acute padj"]

# TBI Subacute vs Subacute Control 분석
subacute_results = run_deseq2_analysis(count_data, col_data, "TBI-Subacute", "TBI-Subacute-Control")
subacute_results.columns = ["TBI Subacute vs SControl Log2 fold change", "Subacute pval", "Subacute padj"]

# TBI Chronic vs Chronic Control 분석
chronic_results = run_deseq2_analysis(count_data, col_data, "TBI-Chronic", "TBI-Chronic-Control")
chronic_results.columns = ["TBI Chronic vs CControl Log2 fold change", "Chronic pval", "Chronic padj"]

# 모든 결과를 하나의 DataFrame으로 병합
final_results = pd.concat([acute_results, subacute_results, chronic_results], axis=1)

# 결과 저장
final_results.to_csv("/data1/project/eunyi/tbi/tbi_deseq2.csv")


Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 3.19 seconds.

Fitting dispersion trend curve...
... done in 0.48 seconds.

Fitting MAP dispersions...
... done in 3.33 seconds.

Fitting LFCs...
... done in 2.76 seconds.

Calculating cook's distance...
... done in 0.02 seconds.

Replacing 0 outlier genes.

Running Wald tests...
... done in 3.39 seconds.



Log2 fold change & Wald test p-value: condition TBI-Acute-Control vs TBI-Acute
                       baseMean  log2FoldChange     lfcSE      stat    pvalue  \
gene                                                                            
ENSMUSG00000000001  5241.281637       -0.273019  0.185252 -1.473774  0.140543   
ENSMUSG00000000003     0.000000             NaN       NaN       NaN       NaN   
ENSMUSG00000000028   126.639640       -0.062120  0.253146 -0.245392  0.806153   
ENSMUSG00000000031    11.517440       -5.061060  1.235661 -4.095832  0.000042   
ENSMUSG00000000037     8.919250        0.040986  0.794642  0.051578  0.958865   
...                         ...             ...       ...       ...       ...   
ENSMUSG00000093370     0.306176       -1.753574  4.331603 -0.404833  0.685601   
ENSMUSG00000093371     0.000000             NaN       NaN       NaN       NaN   
ENSMUSG00000093372     0.000000             NaN       NaN       NaN       NaN   
ENSMUSG00000093373     0.00000

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 3.15 seconds.

Fitting dispersion trend curve...
... done in 0.47 seconds.

/data1/project/eunyi/envs/anaconda3/envs/deseq/lib/python3.9/site-packages/pydeseq2/dds.py:490: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be poorly estimated by the MAD.
  self.fit_dispersion_prior()
Fitting MAP dispersions...
... done in 3.17 seconds.

Fitting LFCs...
... done in 2.59 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 0 outlier genes.

Running Wald tests...
... done in 3.63 seconds.



Log2 fold change & Wald test p-value: condition TBI-Subacute-Control vs TBI-Subacute
                       baseMean  log2FoldChange     lfcSE      stat    pvalue  \
gene                                                                            
ENSMUSG00000000001  4146.918927       -0.049062  0.180745 -0.271440  0.786052   
ENSMUSG00000000003     0.000000             NaN       NaN       NaN       NaN   
ENSMUSG00000000028   147.821402       -0.265132  0.341740 -0.775829  0.437850   
ENSMUSG00000000031    28.162508       -6.411732  1.549879 -4.136923  0.000035   
ENSMUSG00000000037    10.427321       -0.832614  1.092679 -0.761993  0.446064   
...                         ...             ...       ...       ...       ...   
ENSMUSG00000093370     0.000000             NaN       NaN       NaN       NaN   
ENSMUSG00000093371     0.000000             NaN       NaN       NaN       NaN   
ENSMUSG00000093372     0.000000             NaN       NaN       NaN       NaN   
ENSMUSG00000093373     0

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 3.09 seconds.

Fitting dispersion trend curve...
... done in 0.45 seconds.

/data1/project/eunyi/envs/anaconda3/envs/deseq/lib/python3.9/site-packages/pydeseq2/dds.py:490: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be poorly estimated by the MAD.
  self.fit_dispersion_prior()
Fitting MAP dispersions...
... done in 3.14 seconds.

Fitting LFCs...
... done in 2.65 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 0 outlier genes.

Running Wald tests...
... done in 3.54 seconds.



Log2 fold change & Wald test p-value: condition TBI-Chronic-Control vs TBI-Chronic
                       baseMean  log2FoldChange     lfcSE      stat    pvalue  \
gene                                                                            
ENSMUSG00000000001  4670.385515       -0.123083  0.140971 -0.873111  0.382603   
ENSMUSG00000000003     0.000000             NaN       NaN       NaN       NaN   
ENSMUSG00000000028   137.634012        0.381769  0.264474  1.443504  0.148879   
ENSMUSG00000000031     0.912566       -2.950333  3.831297 -0.770061  0.441264   
ENSMUSG00000000037     4.001478        1.088792  1.416485  0.768657  0.442097   
...                         ...             ...       ...       ...       ...   
ENSMUSG00000093370     0.173520       -0.780197  4.995736 -0.156173  0.875897   
ENSMUSG00000093371     0.000000             NaN       NaN       NaN       NaN   
ENSMUSG00000093372     0.000000             NaN       NaN       NaN       NaN   
ENSMUSG00000093373     0.0

In [2]:
acute_results.to_csv("/data1/project/eunyi/tbi/acute_deseq2.csv")
subacute_results.to_csv("/data1/project/eunyi/tbi/subacute_deseq2.csv")
chronic_results.to_csv("/data1/project/eunyi/tbi/chronic_deseq2.csv")

In [3]:
acute_deseq2 = pd.read_csv("/data1/project/eunyi/tbi/acute_deseq2.csv")
subacute_deseq2 = pd.read_csv("/data1/project/eunyi/tbi/subacute_deseq2.csv")
chronic_deseq2 = pd.read_csv("/data1/project/eunyi/tbi/chronic_deseq2.csv")

In [4]:
# 조건에 맞게 데이터 필터링
up_acute = acute_deseq2[(acute_deseq2['TBI Acute vs AControl Log2 fold change'] > 1.00) & (acute_deseq2['Acute padj'] < 0.05)]
# 필터링된 데이터 확인
print(up_acute.shape)

# 조건에 맞게 데이터 필터링
down_acute = acute_deseq2[(acute_deseq2['TBI Acute vs AControl Log2 fold change'] < -0.8) & (acute_deseq2['Acute padj'] < 0.05)]
# 필터링된 데이터 확인
print(down_acute.shape)

# 조건에 맞게 데이터 필터링 
up_subacute = subacute_deseq2[(subacute_deseq2['TBI Subacute vs SControl Log2 fold change'] > 1.00) & (subacute_deseq2['Subacute padj'] < 0.05)]
# 필터링된 데이터 확인
print(up_subacute.shape)

# 조건에 맞게 데이터 필터링
down_subacute = subacute_deseq2[(subacute_deseq2['TBI Subacute vs SControl Log2 fold change'] < -0.8) & (subacute_deseq2['Subacute padj'] < 0.05)]
# 필터링된 데이터 확인
print(down_subacute.shape)

# 조건에 맞게 데이터 필터링
up_chronic = chronic_deseq2[(chronic_deseq2['TBI Chronic vs CControl Log2 fold change'] > 1.00) & (chronic_deseq2['Chronic padj'] < 0.05)]
# 필터링된 데이터 확인
print(up_chronic.shape)

# 조건에 맞게 데이터 필터링
down_chronic = chronic_deseq2[(chronic_deseq2['TBI Chronic vs CControl Log2 fold change'] < -0.8) & (chronic_deseq2['Chronic padj'] < 0.05)]
# 필터링된 데이터 확인
print(down_chronic.shape)

(107, 4)
(485, 4)
(29, 4)
(324, 4)
(0, 4)
(0, 4)


count 데이터 확인

In [5]:
# 카운트 데이터의 샘플 및 유전자 수 확인
print(count_data.shape)
print(col_data['condition'].value_counts())

# 각 조건에 해당하는 샘플 선택이 제대로 이루어졌는지 확인
selected_samples = col_data[(col_data['condition'] == "TBI-Acute") | (col_data['condition'] == "TBI-Acute-Control")].index
print(selected_samples)

# 분석 후 결과의 첫 몇 줄을 확인하여 NaN 값이 많이 포함되어 있는지 체크
print(acute_results.head())


(37681, 16)
condition
TBI-Acute               3
TBI-Acute-Control       3
TBI-Chronic             3
TBI-Subacute            3
TBI-Chronic-Control     2
TBI-Subacute-Control    2
Name: count, dtype: int64
Index(['SRR5292039', 'SRR5292040', 'SRR5292041', 'SRR5292042', 'SRR5292043',
       'SRR5292044'],
      dtype='object')
                    TBI Acute vs AControl Log2 fold change  Acute pval  \
gene                                                                     
ENSMUSG00000000001                               -0.273019    0.140543   
ENSMUSG00000000003                                     NaN         NaN   
ENSMUSG00000000028                               -0.062120    0.806153   
ENSMUSG00000000031                               -5.061060    0.000042   
ENSMUSG00000000037                                0.040986    0.958865   

                    Acute padj  
gene                            
ENSMUSG00000000001    0.507674  
ENSMUSG00000000003         NaN  
ENSMUSG00000000028    0.

In [6]:
print(count_data)

# 각 샘플별로 0의 개수 계산
zeros_per_sample = (count_data == 0).sum(axis=0)
print("Number of zeros per sample:")
print(zeros_per_sample)  

                    SRR5292039  SRR5292040  SRR5292041  SRR5292042  \
gene                                                                 
ENSMUSG00000000001        3640        5261        5522        5271   
ENSMUSG00000000003           0           0           0           0   
ENSMUSG00000000028         113         131         129         137   
ENSMUSG00000000031           1           1           0          36   
ENSMUSG00000000037          11           7           9          13   
...                        ...         ...         ...         ...   
ENSMUSG00000093370           0           0           0           2   
ENSMUSG00000093371           0           0           0           0   
ENSMUSG00000093372           0           0           0           0   
ENSMUSG00000093373           0           0           0           0   
ENSMUSG00000093374           0           0           0           0   

                    SRR5292043  SRR5292044  SRR5292045  SRR5292046  \
gene               

In [7]:
results = pd.read_csv("tbi_deseq2.csv")
print(results.shape)

# NaN 값이 포함된 행의 개수 계산
nan_rows_count = results.isna().any(axis=1).sum()
# NaN 값이 포함된 행의 개수 출력
print(f"NaN 값이 포함된 행의 개수: {nan_rows_count}")

(37681, 10)
NaN 값이 포함된 행의 개수: 23413


cpm 기준빼고 up과 down 개수 구하기

In [9]:
import pandas as pd

# 분석 결과 데이터 불러오기
final_results = pd.read_csv("/data1/project/eunyi/tbi/tbi_deseq2.csv", index_col=0)

# 필터링 기준 정의
up_regulated_criteria_acute = (final_results['TBI Acute vs AControl Log2 fold change'] > 1.00) & (final_results['Acute padj'] < 0.05)
down_regulated_criteria_acute = (final_results['TBI Acute vs AControl Log2 fold change'] < -0.8) & (final_results['Acute padj'] < 0.05)

up_regulated_criteria_subacute = (final_results['TBI Subacute vs SControl Log2 fold change'] > 1.00) & (final_results['Subacute padj'] < 0.05)
down_regulated_criteria_subacute = (final_results['TBI Subacute vs SControl Log2 fold change'] < -0.8) & (final_results['Subacute padj'] < 0.05)

up_regulated_criteria_chronic = (final_results['TBI Chronic vs CControl Log2 fold change'] > 1.00) & (final_results['Chronic padj'] < 0.05)
down_regulated_criteria_chronic = (final_results['TBI Chronic vs CControl Log2 fold change'] < -0.8) & (final_results['Chronic padj'] < 0.05)

# 각 조건별로 필터링
acute_up = final_results[up_regulated_criteria_acute]
acute_down = final_results[down_regulated_criteria_acute]

subacute_up = final_results[up_regulated_criteria_subacute]
subacute_down = final_results[down_regulated_criteria_subacute]

chronic_up = final_results[up_regulated_criteria_chronic]
chronic_down = final_results[down_regulated_criteria_chronic]

# 각 조건별 유전자 수 계산
acute_up_count = len(acute_up)
acute_down_count = len(acute_down)

subacute_up_count = len(subacute_up)
subacute_down_count = len(subacute_down)

chronic_up_count = len(chronic_up)
chronic_down_count = len(chronic_down)

# 전체 합계
total_up = acute_up_count + subacute_up_count + chronic_up_count
total_down = acute_down_count + subacute_down_count + chronic_down_count

# 결과 출력
print(f"Acute Up: {acute_up_count}")
print(f"Acute Down: {acute_down_count}")
print(f"Subacute Up: {subacute_up_count}")
print(f"Subacute Down: {subacute_down_count}")
print(f"Chronic Up: {chronic_up_count}")
print(f"Chronic Down: {chronic_down_count}")
print(f"Total Up: {total_up}")
print(f"Total Down: {total_down}")


Acute Up: 107
Acute Down: 485
Subacute Up: 29
Subacute Down: 324
Chronic Up: 0
Chronic Down: 0
Total Up: 136
Total Down: 809


In [24]:
# 유전자 이름(gene)만 인덱스에서 추출하여 txt 파일로 저장
acute_up_genes = acute_up.index
acute_down_genes = acute_down.index

subacute_up_genes = subacute_up.index
subacute_down_genes = subacute_down.index

chronic_up_genes = chronic_up.index
chronic_down_genes = chronic_down.index

# 유전자 이름만 txt 파일로 저장
acute_up_genes.to_series().to_csv("acute_up.txt", sep="\t", index=False, header=False)
acute_down_genes.to_series().to_csv("acute_down.txt", sep="\t", index=False, header=False)

subacute_up_genes.to_series().to_csv("subacute_up.txt", sep="\t", index=False, header=False)
subacute_down_genes.to_series().to_csv("subacute_down.txt", sep="\t", index=False, header=False)

chronic_up_genes.to_series().to_csv("chronic_up.txt", sep="\t", index=False, header=False)
chronic_down_genes.to_series().to_csv("chronic_down.txt", sep="\t", index=False, header=False)

print("유전자 이름만 각 조건별로 txt 파일로 저장되었습니다.")


유전자 이름만 각 조건별로 txt 파일로 저장되었습니다.


In [27]:
# acute_up gene symbol로 매핑
import pandas as pd
import mygene

# 1. Ensembl Gene ID 파일 불러오기
ensembl_ids_df = pd.read_csv('acute_up.txt', header=None, names=['Ensembl_ID'])

# 2. Ensembl ID 리스트 추출
ensembl_ids = ensembl_ids_df['Ensembl_ID'].tolist()

# 3. MyGene을 사용하여 Ensembl Gene ID를 Gene Symbol로 변환
mg = mygene.MyGeneInfo()
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')

# 4. 매핑 결과를 데이터프레임으로 변환
gene_mapping = {entry['query']: entry.get('symbol', 'NA') for entry in gene_info}
ensembl_ids_df['GeneSymbol'] = ensembl_ids_df['Ensembl_ID'].map(gene_mapping)

# 5. 매핑된 결과를 텍스트 파일로 저장
ensembl_ids_df.to_csv('acute_up_gene.txt', sep='\t', index=False, header=True)

12 input query terms found no hit:	['ENSMUSG00000068869', 'ENSMUSG00000075308', 'ENSMUSG00000078493', 'ENSMUSG00000087156', 'ENSMUSG000


In [28]:
# acute_down gene symbol로 매핑
import pandas as pd
import mygene

# 1. Ensembl Gene ID 파일 불러오기
ensembl_ids_df = pd.read_csv('acute_down.txt', header=None, names=['Ensembl_ID'])

# 2. Ensembl ID 리스트 추출
ensembl_ids = ensembl_ids_df['Ensembl_ID'].tolist()

# 3. MyGene을 사용하여 Ensembl Gene ID를 Gene Symbol로 변환
mg = mygene.MyGeneInfo()
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')

# 4. 매핑 결과를 데이터프레임으로 변환
gene_mapping = {entry['query']: entry.get('symbol', 'NA') for entry in gene_info}
ensembl_ids_df['GeneSymbol'] = ensembl_ids_df['Ensembl_ID'].map(gene_mapping)

# 5. 매핑된 결과를 텍스트 파일로 저장
ensembl_ids_df.to_csv('acute_down_gene.txt', sep='\t', index=False, header=True)

1 input query terms found dup hits:	[('ENSMUSG00000008658', 2)]
17 input query terms found no hit:	['ENSMUSG00000044794', 'ENSMUSG00000069367', 'ENSMUSG00000072566', 'ENSMUSG00000072812', 'ENSMUSG000


In [29]:
# subacute_up gene symbol로 매핑
import pandas as pd
import mygene

# 1. Ensembl Gene ID 파일 불러오기
ensembl_ids_df = pd.read_csv('subacute_up.txt', header=None, names=['Ensembl_ID'])

# 2. Ensembl ID 리스트 추출
ensembl_ids = ensembl_ids_df['Ensembl_ID'].tolist()

# 3. MyGene을 사용하여 Ensembl Gene ID를 Gene Symbol로 변환
mg = mygene.MyGeneInfo()
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')

# 4. 매핑 결과를 데이터프레임으로 변환
gene_mapping = {entry['query']: entry.get('symbol', 'NA') for entry in gene_info}
ensembl_ids_df['GeneSymbol'] = ensembl_ids_df['Ensembl_ID'].map(gene_mapping)

# 5. 매핑된 결과를 텍스트 파일로 저장
ensembl_ids_df.to_csv('subacute_up_gene.txt', sep='\t', index=False, header=True)

9 input query terms found no hit:	['ENSMUSG00000044744', 'ENSMUSG00000059634', 'ENSMUSG00000075930', 'ENSMUSG00000085011', 'ENSMUSG000


In [30]:
# subacute_down gene symbol로 매핑
import pandas as pd
import mygene

# 1. Ensembl Gene ID 파일 불러오기
ensembl_ids_df = pd.read_csv('subacute_down.txt', header=None, names=['Ensembl_ID'])

# 2. Ensembl ID 리스트 추출
ensembl_ids = ensembl_ids_df['Ensembl_ID'].tolist()

# 3. MyGene을 사용하여 Ensembl Gene ID를 Gene Symbol로 변환
mg = mygene.MyGeneInfo()
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')

# 4. 매핑 결과를 데이터프레임으로 변환
gene_mapping = {entry['query']: entry.get('symbol', 'NA') for entry in gene_info}
ensembl_ids_df['GeneSymbol'] = ensembl_ids_df['Ensembl_ID'].map(gene_mapping)

# 5. 매핑된 결과를 텍스트 파일로 저장
ensembl_ids_df.to_csv('subacute_down_gene.txt', sep='\t', index=False, header=True)

20 input query terms found no hit:	['ENSMUSG00000042233', 'ENSMUSG00000042943', 'ENSMUSG00000065771', 'ENSMUSG00000072566', 'ENSMUSG000


In [31]:
# chronic_up gene symbol로 매핑
import pandas as pd
import mygene

# 1. Ensembl Gene ID 파일 불러오기
ensembl_ids_df = pd.read_csv('chronic_up.txt', header=None, names=['Ensembl_ID'])

# 2. Ensembl ID 리스트 추출
ensembl_ids = ensembl_ids_df['Ensembl_ID'].tolist()

# 3. MyGene을 사용하여 Ensembl Gene ID를 Gene Symbol로 변환
mg = mygene.MyGeneInfo()
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')

# 4. 매핑 결과를 데이터프레임으로 변환
gene_mapping = {entry['query']: entry.get('symbol', 'NA') for entry in gene_info}
ensembl_ids_df['GeneSymbol'] = ensembl_ids_df['Ensembl_ID'].map(gene_mapping)

# 5. 매핑된 결과를 텍스트 파일로 저장
ensembl_ids_df.to_csv('chronic_up_gene.txt', sep='\t', index=False, header=True)

6 input query terms found no hit:	['ENSMUSG00000076327', 'ENSMUSG00000085661', 'ENSMUSG00000086292', 'ENSMUSG00000088881', 'ENSMUSG000


In [32]:
# chronic_down gene symbol로 매핑
import pandas as pd
import mygene

# 1. Ensembl Gene ID 파일 불러오기
ensembl_ids_df = pd.read_csv('chronic_down.txt', header=None, names=['Ensembl_ID'])

# 2. Ensembl ID 리스트 추출
ensembl_ids = ensembl_ids_df['Ensembl_ID'].tolist()

# 3. MyGene을 사용하여 Ensembl Gene ID를 Gene Symbol로 변환
mg = mygene.MyGeneInfo()
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')

# 4. 매핑 결과를 데이터프레임으로 변환
gene_mapping = {entry['query']: entry.get('symbol', 'NA') for entry in gene_info}
ensembl_ids_df['GeneSymbol'] = ensembl_ids_df['Ensembl_ID'].map(gene_mapping)

# 5. 매핑된 결과를 텍스트 파일로 저장
ensembl_ids_df.to_csv('chronic_down_gene.txt', sep='\t', index=False, header=True)

5 input query terms found no hit:	['ENSMUSG00000070632', 'ENSMUSG00000076520', 'ENSMUSG00000086988', 'ENSMUSG00000091217', 'ENSMUSG000


In [6]:
# 총 카운트 합 계산 (샘플 별로)
total_counts = count_data.sum(axis=0)

# total_counts에 NaN이나 0이 있는지 확인
print("Total Counts (각 샘플의 총 카운트):")
print(total_counts)

# NaN이나 0이 있는지 확인
print("NaN 또는 0인 총 카운트가 있는지 확인:")
print(total_counts.isna().sum())
print((total_counts == 0).sum())

# count_data의 첫 몇 줄 확인
print("Count Data 예시:")
print(count_data.head())

# count_data에 NaN 값이 있는지 확인
print("Count Data에 NaN 값이 있는지 확인:")
print(count_data.isna().sum())

# count_data에서 모든 값이 0인 유전자가 있는지 확인
print("모든 값이 0인 유전자의 수:")
print((count_data == 0).all(axis=1).sum())


Total Counts (각 샘플의 총 카운트):
SRR5292039    28658575
SRR5292040    31982646
SRR5292041    34407567
SRR5292042    32054780
SRR5292043    33591716
SRR5292044    31180788
SRR5292045    31500774
SRR5292046    29810351
SRR5292047    32766308
SRR5292048    35826173
SRR5292049    34895235
SRR5292050    31570659
SRR5292051    27964934
SRR5292052    28499151
SRR5292053    33398055
SRR5292054    29889782
dtype: int64
NaN 또는 0인 총 카운트가 있는지 확인:
0
0
Count Data 예시:
                    SRR5292039  SRR5292040  SRR5292041  SRR5292042  \
gene                                                                 
ENSMUSG00000000001        3640        5261        5522        5271   
ENSMUSG00000000003           0           0           0           0   
ENSMUSG00000000028         113         131         129         137   
ENSMUSG00000000031           1           1           0          36   
ENSMUSG00000000037          11           7           9          13   

                    SRR5292043  SRR5292044  SRR5292045  

In [11]:
import pandas as pd
import glob

# 샘플 리스트 및 조건 매핑
samples = ["SRR5292039", "SRR5292040", "SRR5292041", "SRR5292042", 
           "SRR5292043", "SRR5292044", "SRR5292045", "SRR5292046", 
           "SRR5292047", "SRR5292048", "SRR5292049", "SRR5292050", 
           "SRR5292051", "SRR5292052", "SRR5292053", "SRR5292054"]

conditions = ["TBI_acute_Control", "TBI_acute_Control", "TBI_acute_Control", 
              "TBI_acute", "TBI_acute", "TBI_acute",
              "TBI_Subacute_Control", "TBI_Subacute_Control", 
              "TBI_subacute", "TBI_subacute", "TBI_subacute", 
              "TBI_chronic_control", "TBI_chronic_control", 
              "TBI_Chronic", "TBI_Chronic", "TBI_Chronic"]

# 1. CPM 기반 필터링 (기존 코드 그대로)
file_pattern = '/data1/project/eunyi/tbi/data/htseq_counts/*_counts.txt'
cpm_filter_df = pd.DataFrame()

for file_path in glob.glob(file_pattern):
    # 파일 읽기
    sample_name = file_path.split("/")[-1].replace("_counts.txt", "")
    counts_df = pd.read_csv(file_path, sep="\t", header=None, names=["gene", "raw_count"])
    
    # 특수 항목 제거
    counts_df = counts_df[~counts_df['gene'].str.startswith('__')]
    
    # 총 읽기 수 계산
    total_counts = counts_df["raw_count"].sum()
    
    # CPM 계산
    counts_df["CPM"] = (counts_df["raw_count"] / total_counts) * 1e6
    
    # 유전자 이름을 인덱스로 설정
    counts_df.set_index("gene", inplace=True)
    
    # CPM 데이터만 추출
    cpm_df = counts_df[["CPM"]]
    cpm_df.columns = [sample_name]
    
    # 통합 데이터프레임에 병합
    if cpm_filter_df.empty:
        cpm_filter_df = cpm_df
    else:
        cpm_filter_df = cpm_filter_df.join(cpm_df, how='outer')

# 모든 샘플에서 CPM이 5 이상인 유전자만 필터링
filtered_genes = cpm_filter_df[(cpm_filter_df >= 5).all(axis=1)].index

# 2. 필터링된 유전자에 대한 raw count 데이터 추출
count_data = pd.DataFrame()

for sample in samples:
    sample_data = pd.read_csv(f"/data1/project/eunyi/tbi/data/htseq_counts/{sample}_counts.txt", sep="\t", header=None, index_col=0)
    
    # 특수 항목 제거
    sample_data = sample_data[~sample_data.index.str.startswith("__")]
    
    # 필터링된 유전자만 선택
    sample_data = sample_data.loc[filtered_genes]
    
    count_data[sample] = sample_data[1]

# gene 열을 인덱스로 설정
count_data.index.name = "gene"

# 샘플 조건을 DataFrame으로 변환
col_data = pd.DataFrame({'condition': conditions}, index=samples)
col_data['condition'] = pd.Categorical(col_data['condition'])

# 3. DESeq2 분석
def run_deseq2_analysis(count_data, col_data, condition1, condition2):
    # 해당 조건에 해당하는 샘플만 선택
    selected_samples = col_data[(col_data['condition'] == condition1) | (col_data['condition'] == condition2)].index
    selected_count_data = count_data[selected_samples]
    selected_col_data = col_data.loc[selected_samples]
    
    # DESeq2 분석 수행
    dds = DeseqDataSet(counts=selected_count_data.T, metadata=selected_col_data, design_factors="condition", refit_cooks=True, n_cpus=8)
    dds.deseq2()
    deseq_stats = DeseqStats(dds)
    deseq_stats.summary()
    
    # 결과 반환
    return deseq_stats.results_df[['log2FoldChange', 'pvalue', 'padj']]

# TBI Acute vs Acute Control 분석
acute_results = run_deseq2_analysis(count_data, col_data, "TBI_acute", "TBI_acute_Control")
acute_results.columns = ["TBI Acute vs AControl Log2 fold change", "Acute pval", "Acute padj"]

# TBI Subacute vs Subacute Control 분석
subacute_results = run_deseq2_analysis(count_data, col_data, "TBI_subacute", "TBI_Subacute_Control")
subacute_results.columns = ["TBI Subacute vs SControl Log2 fold change", "Subacute pval", "Subacute padj"]

# TBI Chronic vs Chronic Control 분석
chronic_results = run_deseq2_analysis(count_data, col_data, "TBI_Chronic", "TBI_chronic_control")
chronic_results.columns = ["TBI Chronic vs CControl Log2 fold change", "Chronic pval", "Chronic padj"]

# 모든 결과를 하나의 DataFrame으로 병합
final_results = pd.concat([acute_results, subacute_results, chronic_results], axis=1)

# 결과 저장
final_results.to_csv("/data1/project/eunyi/tbi/tbi_deseq2_cpm.csv")


/data1/project/eunyi/envs/anaconda3/envs/deseq/lib/python3.9/site-packages/pydeseq2/dds.py:264: UserWarning: Some factor levels in the design contain underscores ('_').
                    They will be converted to hyphens ('-').
  self.obsm["design_matrix"] = build_design_matrix(
Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 1.11 seconds.

Fitting dispersion trend curve...
/data1/project/eunyi/envs/anaconda3/envs/deseq/lib/python3.9/site-packages/pydeseq2/dds.py:716: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
... done in 0.14 seconds.

Fitting MAP dispersions...
... done in 1.21 seconds.

Fitting LFCs...
... done in 0.85 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 0 outlier genes.

Running Wald tests...
... done in 2.08 seconds.



Log2 fold change & Wald test p-value: condition TBI-acute-Control vs TBI-acute
                        baseMean  log2FoldChange     lfcSE      stat  \
gene                                                                   
ENSMUSG00000000001   5272.282107       -0.280304  0.190522 -1.471244   
ENSMUSG00000000056   1004.343582        0.024106  0.195688  0.123185   
ENSMUSG00000000058   3605.929661        0.518505  0.178390  2.906577   
ENSMUSG00000000078  42177.548315        0.116703  0.219614  0.531401   
ENSMUSG00000000085   1683.592477        0.141013  0.188349  0.748683   
...                          ...             ...       ...       ...   
ENSMUSG00000092416    367.422233        0.271246  0.212413  1.276975   
ENSMUSG00000092558    842.045287        0.192205  0.165550  1.161009   
ENSMUSG00000092607    334.323917        0.278969  0.193464  1.441971   
ENSMUSG00000093077   9744.784294        1.129989  0.251349  4.495695   
ENSMUSG00000093098  26550.827556        0.557335  0.20756

/data1/project/eunyi/envs/anaconda3/envs/deseq/lib/python3.9/site-packages/pydeseq2/dds.py:264: UserWarning: Some factor levels in the design contain underscores ('_').
                    They will be converted to hyphens ('-').
  self.obsm["design_matrix"] = build_design_matrix(
Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 1.12 seconds.

Fitting dispersion trend curve...
/data1/project/eunyi/envs/anaconda3/envs/deseq/lib/python3.9/site-packages/pydeseq2/dds.py:716: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
... done in 0.14 seconds.

/data1/project/eunyi/envs/anaconda3/envs/deseq/lib/python3.9/site-packages/pydeseq2/dds.py:490: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be poorly estimated by the MAD.
  self.fit_dispersion_prior()
Fitting 

Log2 fold change & Wald test p-value: condition TBI-subacute vs TBI-Subacute-Control
                        baseMean  log2FoldChange     lfcSE      stat  \
gene                                                                   
ENSMUSG00000000001   4167.658257        0.081709  0.179497  0.455210   
ENSMUSG00000000056    807.544017        0.050519  0.200255  0.252273   
ENSMUSG00000000058   3687.667306       -0.250337  0.193689 -1.292472   
ENSMUSG00000000078  32195.107283       -0.044379  0.192743 -0.230250   
ENSMUSG00000000085   1827.994546        0.365836  0.182456  2.005065   
...                          ...             ...       ...       ...   
ENSMUSG00000092416    382.387751       -0.241889  0.238089 -1.015958   
ENSMUSG00000092558    871.642560       -0.168781  0.196400 -0.859371   
ENSMUSG00000092607    363.898016       -0.101530  0.202448 -0.501512   
ENSMUSG00000093077  14441.873493       -1.252231  0.294330 -4.254510   
ENSMUSG00000093098  43490.091747       -1.050084  0

/data1/project/eunyi/envs/anaconda3/envs/deseq/lib/python3.9/site-packages/pydeseq2/dds.py:264: UserWarning: Some factor levels in the design contain underscores ('_').
                    They will be converted to hyphens ('-').
  self.obsm["design_matrix"] = build_design_matrix(
Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 1.19 seconds.

Fitting dispersion trend curve...
... done in 0.15 seconds.

/data1/project/eunyi/envs/anaconda3/envs/deseq/lib/python3.9/site-packages/pydeseq2/dds.py:490: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be poorly estimated by the MAD.
  self.fit_dispersion_prior()
Fitting MAP dispersions...
... done in 1.14 seconds.

Fitting LFCs...
... done in 0.82 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 0 outlier genes.

Running Wald tests...


Log2 fold change & Wald test p-value: condition TBI-chronic-control vs TBI-Chronic
                        baseMean  log2FoldChange     lfcSE      stat  \
gene                                                                   
ENSMUSG00000000001   4684.124557       -0.104235  0.143972 -0.723998   
ENSMUSG00000000056    693.806043       -0.043247  0.145558 -0.297116   
ENSMUSG00000000058   4212.414778       -0.101298  0.130123 -0.778476   
ENSMUSG00000000078  36527.001268        0.130132  0.177634  0.732585   
ENSMUSG00000000085   1724.333037        0.023591  0.137408  0.171686   
...                          ...             ...       ...       ...   
ENSMUSG00000092416    322.023155        0.065298  0.196328  0.332599   
ENSMUSG00000092558    785.776115        0.006427  0.158432  0.040569   
ENSMUSG00000092607    310.524515       -0.302798  0.213265 -1.419821   
ENSMUSG00000093077  10859.033785        0.644249  0.186681  3.451075   
ENSMUSG00000093098  40649.537914        0.337133  0.4

... done in 2.03 seconds.



cpm 기준 적용 deg선별

In [13]:
import pandas as pd

# DESeq2 분석 결과 불러오기
results_file = "/data1/project/eunyi/tbi/tbi_deseq2_cpm.csv"
results_df = pd.read_csv(results_file, index_col=0)

# p-value 임계값 설정
pvalue_threshold = 0.05

# 각 조건별로 Up/Down 유전자 개수 계산
def count_up_down_genes(results_df, log2fc_col, padj_col, up_threshold, down_threshold):
    upregulated = results_df[(results_df[log2fc_col] > up_threshold) & (results_df[padj_col] < pvalue_threshold)]
    downregulated = results_df[(results_df[log2fc_col] < down_threshold) & (results_df[padj_col] < pvalue_threshold)]
    return len(upregulated), len(downregulated)

# Acute vs Control
acute_up, acute_down = count_up_down_genes(results_df, "TBI Acute vs AControl Log2 fold change", "Acute padj", 1.00, -0.8)

# Subacute vs Control
subacute_up, subacute_down = count_up_down_genes(results_df, "TBI Subacute vs SControl Log2 fold change", "Subacute padj", 1.00, -0.8)

# Chronic vs Control
chronic_up, chronic_down = count_up_down_genes(results_df, "TBI Chronic vs CControl Log2 fold change", "Chronic padj", 1.00, -0.8)

# Total up and down
total_up = acute_up + subacute_up + chronic_up
total_down = acute_down + subacute_down + chronic_down

# 결과 출력
print(f"Acute: Up = {acute_up}, Down = {acute_down}")
print(f"Subacute: Up = {subacute_up}, Down = {subacute_down}")
print(f"Chronic: Up = {chronic_up}, Down = {chronic_down}")
print(f"Total: Up = {total_up}, Down = {total_down}")


Acute: Up = 42, Down = 143
Subacute: Up = 28, Down = 34
Chronic: Up = 0, Down = 0
Total: Up = 70, Down = 177


In [44]:
# acute_up_cpm gene symbol로 매핑
import pandas as pd
import mygene

# 1. Ensembl Gene ID 파일 불러오기
ensembl_ids_df = pd.read_csv('acute_up_cpm.txt', header=None, names=['Ensembl_ID'])

# 2. Ensembl ID 리스트 추출
ensembl_ids = ensembl_ids_df['Ensembl_ID'].tolist()

# 3. MyGene을 사용하여 Ensembl Gene ID를 Gene Symbol로 변환
mg = mygene.MyGeneInfo()
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')

# 4. 매핑 결과를 데이터프레임으로 변환
gene_mapping = {entry['query']: entry.get('symbol', 'NA') for entry in gene_info}
ensembl_ids_df['GeneSymbol'] = ensembl_ids_df['Ensembl_ID'].map(gene_mapping)

# 5. 매핑된 결과를 텍스트 파일로 저장
ensembl_ids_df.to_csv('acute_up_cpm_gene.txt', sep='\t', index=False, header=True)

1 input query terms found no hit:	['ENSMUSG00000093077']


In [45]:
# acute_down_cpm gene symbol로 매핑
import pandas as pd
import mygene

# 1. Ensembl Gene ID 파일 불러오기
ensembl_ids_df = pd.read_csv('acute_down_cpm.txt', header=None, names=['Ensembl_ID'])

# 2. Ensembl ID 리스트 추출
ensembl_ids = ensembl_ids_df['Ensembl_ID'].tolist()

# 3. MyGene을 사용하여 Ensembl Gene ID를 Gene Symbol로 변환
mg = mygene.MyGeneInfo()
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')

# 4. 매핑 결과를 데이터프레임으로 변환
gene_mapping = {entry['query']: entry.get('symbol', 'NA') for entry in gene_info}
ensembl_ids_df['GeneSymbol'] = ensembl_ids_df['Ensembl_ID'].map(gene_mapping)

# 5. 매핑된 결과를 텍스트 파일로 저장
ensembl_ids_df.to_csv('acute_down_cpm_gene.txt', sep='\t', index=False, header=True)

2 input query terms found no hit:	['ENSMUSG00000044794', 'ENSMUSG00000069367']


In [46]:
# subacute_up_cpm gene symbol로 매핑
import pandas as pd
import mygene

# 1. Ensembl Gene ID 파일 불러오기
ensembl_ids_df = pd.read_csv('subacute_up_cpm.txt', header=None, names=['Ensembl_ID'])

# 2. Ensembl ID 리스트 추출
ensembl_ids = ensembl_ids_df['Ensembl_ID'].tolist()

# 3. MyGene을 사용하여 Ensembl Gene ID를 Gene Symbol로 변환
mg = mygene.MyGeneInfo()
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')

# 4. 매핑 결과를 데이터프레임으로 변환
gene_mapping = {entry['query']: entry.get('symbol', 'NA') for entry in gene_info}
ensembl_ids_df['GeneSymbol'] = ensembl_ids_df['Ensembl_ID'].map(gene_mapping)

# 5. 매핑된 결과를 텍스트 파일로 저장
ensembl_ids_df.to_csv('subacute_up_cpm_gene.txt', sep='\t', index=False, header=True)

In [47]:
# subacute_down_cpm gene symbol로 매핑
import pandas as pd
import mygene

# 1. Ensembl Gene ID 파일 불러오기
ensembl_ids_df = pd.read_csv('subacute_down_cpm.txt', header=None, names=['Ensembl_ID'])

# 2. Ensembl ID 리스트 추출
ensembl_ids = ensembl_ids_df['Ensembl_ID'].tolist()

# 3. MyGene을 사용하여 Ensembl Gene ID를 Gene Symbol로 변환
mg = mygene.MyGeneInfo()
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')

# 4. 매핑 결과를 데이터프레임으로 변환
gene_mapping = {entry['query']: entry.get('symbol', 'NA') for entry in gene_info}
ensembl_ids_df['GeneSymbol'] = ensembl_ids_df['Ensembl_ID'].map(gene_mapping)

# 5. 매핑된 결과를 텍스트 파일로 저장
ensembl_ids_df.to_csv('subacute_down_cpm_gene.txt', sep='\t', index=False, header=True)

2 input query terms found no hit:	['ENSMUSG00000093077', 'ENSMUSG00000093098']


In [48]:
# chronic_down_cpm gene symbol로 매핑
import pandas as pd
import mygene

# 1. Ensembl Gene ID 파일 불러오기
ensembl_ids_df = pd.read_csv('chronic_down_cpm.txt', header=None, names=['Ensembl_ID'])

# 2. Ensembl ID 리스트 추출
ensembl_ids = ensembl_ids_df['Ensembl_ID'].tolist()

# 3. MyGene을 사용하여 Ensembl Gene ID를 Gene Symbol로 변환
mg = mygene.MyGeneInfo()
gene_info = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='mouse')

# 4. 매핑 결과를 데이터프레임으로 변환
gene_mapping = {entry['query']: entry.get('symbol', 'NA') for entry in gene_info}
ensembl_ids_df['GeneSymbol'] = ensembl_ids_df['Ensembl_ID'].map(gene_mapping)

# 5. 매핑된 결과를 텍스트 파일로 저장
ensembl_ids_df.to_csv('chronic_down_cpm_gene.txt', sep='\t', index=False, header=True)